In [1]:
!pip install redis
!pip install pymysql

In [12]:
import redis, json, time, pymysql
DB_Redis = redis.Redis(host='localhost', port=6379, db=0)
DB_MYSQL = pymysql.connect("localhost","tdr","tdr", "tdr")

def empty_mysql():
    cur = DB_MYSQL.cursor()
    cur.execute('DELETE FROM films WHERE 1=1')
    DB_MYSQL.commit()
    cur.close()

cur = DB_MYSQL.cursor()
cur.execute("INSERT INTO films (title,year,genre_list,cast_list) VALUES('foo', 2019, 'bar', 'baz')")
DB_MYSQL.commit()
cur.close()
cur = DB_MYSQL.cursor()
cur.execute('SELECT title,year,genre_list,cast_list FROM films')
print(cur.fetchall())
cur.close()
empty_mysql()
print(DB_Redis.set('foo','bar'))
print(DB_Redis.get('foo'))
print(DB_Redis.delete('foo'))
print(DB_Redis.get('foo'))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [8]:
with open('movies.json') as f:
    datas_json = json.load(f)
print(datas_json[0])

{'title': 'After Dark in Central Park', 'year': 1900, 'cast': [], 'genres': []}


In [9]:
t = time.time()
for film in datas_json:
    DB_Redis.set(film['title'], json.dumps(film))
t = time.time() - t
print('Insertions de {} films dans Redis : {:0.03f} secondes'.format(len(datas_json), t))
#avatar = json.loads(DB_Redis.get('Avatar'))
#print(avatar['year'])

Insertions de 28795 films dans Redis : 2.004 secondes


In [11]:
empty_mysql()
datas_tuple = []
for film in datas_json:
    datas_tuple.append((film['title'],str(film['year']),json.dumps(film['genres']),json.dumps(film['cast'])))
print(datas_tuple[0])
t = time.time()
sql = "INSERT INTO films(title,year,genre_list,cast_list) VALUES(%s,%s,%s,%s)"
try:
    # create a new cursor
    cur = DB_MYSQL.cursor()
    # execute the INSERT statement
    cur.executemany(sql,datas_tuple)
    # commit the changes to the database
    DB_MYSQL.commit()
    # close communication with the database
    cur.close()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
t = time.time() - t
print('Insertions de {} films dans MySQL : {:0.03f} secondes'.format(len(datas_json), t))
cur = DB_MYSQL.cursor()
cur.execute('SELECT title,year,genre_list,cast_list FROM films')
print('{} films dans MySQL'.format(cur.rowcount))
cur.close()

('After Dark in Central Park', '1900', '[]', '[]')
Insertions de 28795 films dans MySQL : 0.620 secondes
57590 films dans MySQL
